# Modeling

In [1]:
from datetime import datetime
start_time = datetime.now()

# Libraries to load
# !pip install lightgbm

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import lightgbm as lgb
import numpy as np
%matplotlib inline

print('LOAD DURATION: ', datetime.now() - start_time) # load time about 30 seconds

LOAD DURATION:  0:00:01.663038


In [2]:
%%time
vars = pd.read_csv('vars_final_HW11.csv')
print(vars.shape)
vars.head()

(96397, 22)
CPU times: user 214 ms, sys: 28.8 ms, total: 243 ms
Wall time: 249 ms


,card_zip3_total_7,merchnum_state_total_1,Cardnum_total_3,card_merch_max_3,card_zip_total_30,card_zip_max_45,card_zip_total_0,card_merch_max_30,card_zip3_total_0,card_merch_total_0,...,card_state_total_14,card_zip_max_3,merchnum_zip3_total_1,card_state_total_7,card_zip3_total_30,card_state_total_45,card_state_max_3,card_state_max_1,Recnum,Fraud
0,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,...,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,1,0
1,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,...,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,2,0
2,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,...,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,3,0
3,3.62,7.24,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,...,3.62,3.62,7.24,3.62,3.62,3.62,3.62,3.62,4,0
4,7.24,10.86,7.24,3.62,7.24,3.62,7.24,3.62,7.24,7.24,...,7.24,3.62,10.86,7.24,7.24,7.24,3.62,3.62,5,0


In [3]:
final_vars = pd.read_csv('final_vars_list.csv')
final_vars

,variable name
0,card_zip3_total_7
1,merchnum_state_total_1
2,Cardnum_total_3
3,card_merch_max_3
4,card_zip_total_30
5,card_zip_max_45
6,card_zip_total_0
7,card_merch_max_30
8,card_zip3_total_0
9,card_merch_total_0


In [4]:
numvars = min(15,len(final_vars))
final_vars_list = ['Recnum','Fraud']
for i in range(numvars):
    final_vars_list.append(final_vars.iloc[i]['variable name'])
    
final_vars_list

['Recnum',
 'Fraud',
 'card_zip3_total_7',
 'merchnum_state_total_1',
 'Cardnum_total_3',
 'card_merch_max_3',
 'card_zip_total_30',
 'card_zip_max_45',
 'card_zip_total_0',
 'card_merch_max_30',
 'card_zip3_total_0',
 'card_merch_total_0',
 'card_state_total_0',
 'card_state_total_30',
 'card_state_total_14',
 'card_zip_max_3',
 'merchnum_zip3_total_1']

In [5]:
vars = vars.filter(final_vars_list,axis=1)
vars.head()

,Recnum,Fraud,card_zip3_total_7,merchnum_state_total_1,Cardnum_total_3,card_merch_max_3,card_zip_total_30,card_zip_max_45,card_zip_total_0,card_merch_max_30,card_zip3_total_0,card_merch_total_0,card_state_total_0,card_state_total_30,card_state_total_14,card_zip_max_3,merchnum_zip3_total_1
0,1,0,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62
1,2,0,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42
2,3,0,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49
3,4,0,3.62,7.24,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,7.24
4,5,0,7.24,10.86,7.24,3.62,7.24,3.62,7.24,3.62,7.24,7.24,7.24,7.24,7.24,3.62,10.86


In [6]:
vars.shape

(96397, 17)

In [7]:
# use this to cap variables. For some problems it helps
Clip = 10

In [8]:
# vars.rename(columns={'fraud_label':'Fraud'},inplace=True)

In [9]:
vars['Fraud'].sum()

1059

In [10]:
vars.head(10)

,Recnum,Fraud,card_zip3_total_7,merchnum_state_total_1,Cardnum_total_3,card_merch_max_3,card_zip_total_30,card_zip_max_45,card_zip_total_0,card_merch_max_30,card_zip3_total_0,card_merch_total_0,card_state_total_0,card_state_total_30,card_state_total_14,card_zip_max_3,merchnum_zip3_total_1
0,1,0,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62
1,2,0,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42
2,3,0,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49
3,4,0,3.62,7.24,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,7.24
4,5,0,7.24,10.86,7.24,3.62,7.24,3.62,7.24,3.62,7.24,7.24,7.24,7.24,7.24,3.62,10.86
5,6,0,3.67,14.53,3.67,3.67,3.67,3.67,3.67,3.67,3.67,3.67,3.67,3.67,3.67,3.67,14.53
6,7,0,3.62,18.15,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,18.15
7,8,0,230.32,230.32,230.32,230.32,230.32,230.32,230.32,230.32,230.32,230.32,230.32,230.32,230.32,230.32,230.32
8,9,0,62.11,62.11,62.11,62.11,62.11,62.11,62.11,62.11,62.11,62.11,62.11,62.11,62.11,62.11,62.11
9,10,0,10.86,21.77,10.86,3.62,10.86,3.62,10.86,3.62,10.86,10.86,10.86,10.86,10.86,3.62,21.77


In [11]:
vars.shape

(96397, 17)

In [12]:
vars.describe()

,Recnum,Fraud,card_zip3_total_7,merchnum_state_total_1,Cardnum_total_3,card_merch_max_3,card_zip_total_30,card_zip_max_45,card_zip_total_0,card_merch_max_30,card_zip3_total_0,card_merch_total_0,card_state_total_0,card_state_total_30,card_state_total_14,card_zip_max_3,merchnum_zip3_total_1
count,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.0000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000
mean,48365.481820,0.010986,767.821198,1133.001594,1512.932952,441.556293,992.1012,562.502062,531.796481,515.250618,538.522017,529.059538,553.961983,1664.710003,1149.646137,446.467873,1144.441794
std,27945.003883,0.104236,4138.019507,4376.722457,6115.505329,1014.692469,4346.6019,1104.884808,2623.087903,1079.857559,2626.537992,2622.052105,2640.513565,5178.663648,4505.749843,1018.281875,4387.537848
min,1.000000,0.000000,0.010000,0.010000,0.010000,0.010000,0.0100,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000
25%,24154.000000,0.000000,77.400000,118.000000,140.000000,40.000000,105.7500,64.940000,47.330000,53.100000,48.040000,46.620000,49.120000,166.990000,113.470000,41.210000,117.700000
50%,48365.000000,0.000000,238.800000,396.530000,518.610000,157.000000,322.7800,235.930000,160.940000,200.000000,164.500000,159.400000,169.000000,538.350000,365.050000,160.000000,395.610000
75%,72578.000000,0.000000,695.000000,1141.660000,1550.000000,480.000000,906.2700,670.000000,495.900000,591.900000,500.000000,494.000000,514.680000,1604.900000,1090.670000,487.640000,1148.330000
max,96753.000000,1.000000,306633.410000,306633.410000,310843.060000,47900.000000,306633.4100,47900.000000,217467.180000,47900.000000,217467.180000,217467.180000,217467.180000,306633.410000,306633.410000,47900.000000,306633.410000


In [13]:
record_save = vars['Recnum']
Y_save = pd.DataFrame(vars.loc[:,'Fraud'])
Y_save.head()

,Fraud
0,0
1,0
2,0
3,0
4,0


## Scale and truncate field values

In [14]:
X_no_scaling = vars.drop(columns = ['Recnum','Fraud'])
X_no_scaling.describe()

,card_zip3_total_7,merchnum_state_total_1,Cardnum_total_3,card_merch_max_3,card_zip_total_30,card_zip_max_45,card_zip_total_0,card_merch_max_30,card_zip3_total_0,card_merch_total_0,card_state_total_0,card_state_total_30,card_state_total_14,card_zip_max_3,merchnum_zip3_total_1
count,96397.000000,96397.000000,96397.000000,96397.000000,96397.0000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000
mean,767.821198,1133.001594,1512.932952,441.556293,992.1012,562.502062,531.796481,515.250618,538.522017,529.059538,553.961983,1664.710003,1149.646137,446.467873,1144.441794
std,4138.019507,4376.722457,6115.505329,1014.692469,4346.6019,1104.884808,2623.087903,1079.857559,2626.537992,2622.052105,2640.513565,5178.663648,4505.749843,1018.281875,4387.537848
min,0.010000,0.010000,0.010000,0.010000,0.0100,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000
25%,77.400000,118.000000,140.000000,40.000000,105.7500,64.940000,47.330000,53.100000,48.040000,46.620000,49.120000,166.990000,113.470000,41.210000,117.700000
50%,238.800000,396.530000,518.610000,157.000000,322.7800,235.930000,160.940000,200.000000,164.500000,159.400000,169.000000,538.350000,365.050000,160.000000,395.610000
75%,695.000000,1141.660000,1550.000000,480.000000,906.2700,670.000000,495.900000,591.900000,500.000000,494.000000,514.680000,1604.900000,1090.670000,487.640000,1148.330000
max,306633.410000,306633.410000,310843.060000,47900.000000,306633.4100,47900.000000,217467.180000,47900.000000,217467.180000,217467.180000,217467.180000,306633.410000,306633.410000,47900.000000,306633.410000


In [15]:
X = (X_no_scaling - X_no_scaling.mean()) / X_no_scaling.std()

In [16]:
# push in any outlier values
cols = X.columns
X.loc[:,cols] = X[cols].clip(upper=Clip)
X.loc[:,cols] = X[cols].clip(lower=-1*Clip)
# X = (X_no_scaling - X_no_scaling.mean()) / X_no_scaling.std()
X.describe()

,card_zip3_total_7,merchnum_state_total_1,Cardnum_total_3,card_merch_max_3,card_zip_total_30,card_zip_max_45,card_zip_total_0,card_merch_max_30,card_zip3_total_0,card_merch_total_0,card_state_total_0,card_state_total_30,card_state_total_14,card_zip_max_3,merchnum_zip3_total_1
count,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000,96397.000000
mean,-0.012379,-0.011511,-0.016033,-0.011808,-0.011516,-0.011062,-0.011770,-0.011796,-0.011743,-0.011778,-0.011640,-0.009957,-0.011546,-0.011722,-0.011496
std,0.478227,0.561196,0.603512,0.755562,0.554095,0.790978,0.483555,0.768307,0.485917,0.482831,0.495104,0.707512,0.585563,0.758208,0.563914
min,-0.185550,-0.258868,-0.247391,-0.435153,-0.228245,-0.509096,-0.202733,-0.477138,-0.205027,-0.201769,-0.209789,-0.321454,-0.255149,-0.438442,-0.260837
25%,-0.166848,-0.231909,-0.224500,-0.395742,-0.203918,-0.450329,-0.184693,-0.427974,-0.186741,-0.183993,-0.191191,-0.289210,-0.229968,-0.397982,-0.234013
50%,-0.127844,-0.168270,-0.162590,-0.280436,-0.153987,-0.295571,-0.141382,-0.291937,-0.142401,-0.140981,-0.145791,-0.217500,-0.174132,-0.281325,-0.170672
75%,-0.017598,0.001978,0.006061,0.037887,-0.019747,0.097293,-0.013685,0.070981,-0.014666,-0.013371,-0.014877,-0.011549,-0.013089,0.040433,0.000886
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [17]:
# separate data into modeling (traintest) and out of time
oot_recnum=84300
X_trntst = X[0:oot_recnum]
Y_trntst = Y_save[0:oot_recnum]
X_oot = X[oot_recnum:]
Y_oot = Y_save[oot_recnum:]

In [18]:
niter = 0
nitermax = 10
FDR3 = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
X_oot_orig = X_oot.copy()
pd.options.mode.chained_assignment = None  # default='warn'

## Models. You can comment out any of these cells and just explore one model type. You can also just rerun that single cell multiple times as you explore different model hyperparameters.

In [19]:
%%time
# Logistic regression

for niter in range(nitermax):    
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

    model = LogisticRegression(max_iter=1000)
 
    X_oot = X_oot_orig.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()

    model.fit(X_trn, Y_trn.values.ravel())   

    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['Fraud'] = Y_trn_save['Fraud']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['Fraud'] = Y_tst['Fraud']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions
    X_oot['Fraud'] = Y_oot['Fraud']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 0.6035031847133758 0.6428571428571429 0.29608938547486036
1 0.636650868878357 0.6194331983805668 0.29608938547486036
2 0.6218487394957983 0.6175438596491228 0.29608938547486036
3 0.6148969889064976 0.642570281124498 0.29608938547486036
4 0.6307448494453248 0.5983935742971888 0.29608938547486036
5 0.6252072968490879 0.5848375451263538 0.29608938547486036
6 0.602291325695581 0.6171003717472119 0.3016759776536313
7 0.6015625 0.65 0.30726256983240224
8 0.6237623762376238 0.5802919708029197 0.29608938547486036
9 0.6245954692556634 0.5763358778625954 0.3016759776536313
trn    0.618506
tst    0.612936
oot    0.298324
dtype: float64
CPU times: user 6.31 s, sys: 39 ms, total: 6.35 s
Wall time: 6.35 s


In [20]:
%%time
# Single DT

for niter in range(nitermax):    
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

    model = DecisionTreeClassifier()
 
    X_oot = X_oot_orig.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()

    model.fit(X_trn, Y_trn.values.ravel())   

    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['Fraud'] = Y_trn_save['Fraud']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['Fraud'] = Y_tst['Fraud']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions
    X_oot['Fraud'] = Y_oot['Fraud']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 1.0 0.6475095785440613 0.2346368715083799
1 1.0 0.6639004149377593 0.20670391061452514
2 1.0 0.6465863453815262 0.2737430167597765
3 1.0 0.5772058823529411 0.24022346368715083
4 1.0 0.5610687022900763 0.16201117318435754
5 1.0 0.622568093385214 0.2849162011173184
6 1.0 0.6200873362445415 0.2122905027932961
7 1.0 0.5953307392996109 0.2122905027932961
8 1.0 0.6182572614107884 0.2346368715083799
9 1.0 0.6185567010309279 0.22346368715083798
trn    1.000000
tst    0.617107
oot    0.228492
dtype: float64
CPU times: user 17.3 s, sys: 28.4 ms, total: 17.4 s
Wall time: 17.4 s


In [21]:
%%time
# RF

for niter in range(nitermax):    
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

    model = RandomForestClassifier()
    X_oot = X_oot_orig.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()

    model.fit(X_trn, Y_trn.values.ravel())   

    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['Fraud'] = Y_trn_save['Fraud']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['Fraud'] = Y_tst['Fraud']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions
    X_oot['Fraud'] = Y_oot['Fraud']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 1.0 0.8284671532846716 0.41899441340782123
1 1.0 0.7837837837837838 0.3463687150837989
2 1.0 0.782051282051282 0.3575418994413408
3 1.0 0.8091603053435115 0.36312849162011174
4 1.0 0.8140495867768595 0.3463687150837989
5 1.0 0.8060836501901141 0.35195530726256985
6 1.0 0.8264150943396227 0.3687150837988827
7 1.0 0.8194945848375451 0.3575418994413408
8 1.0 0.8407407407407408 0.33519553072625696
9 1.0 0.8315018315018315 0.3463687150837989
trn    1.000000
tst    0.814175
oot    0.359218
dtype: float64
CPU times: user 3min 5s, sys: 51.6 ms, total: 3min 5s
Wall time: 3min 5s


In [22]:
%%time
# BT

for niter in range(nitermax):    
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

    model = lgb.LGBMClassifier()

    X_oot = X_oot_orig.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()

    model.fit(X_trn, Y_trn.values.ravel())   

    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['Fraud'] = Y_trn_save['Fraud']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['Fraud'] = Y_tst['Fraud']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions
    X_oot['Fraud'] = Y_oot['Fraud']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 0.9936305732484076 0.8253968253968254 0.329608938547486
1 0.9921259842519685 0.7836734693877551 0.33519553072625696
2 0.9935483870967742 0.8 0.29608938547486036
3 0.9935064935064936 0.7803030303030303 0.4022346368715084
4 0.9933554817275747 0.8093525179856115 0.33519553072625696
5 0.9918831168831169 0.8295454545454546 0.31843575418994413
6 0.9902120717781403 0.850187265917603 0.2849162011173184
7 0.9950576606260296 0.8315018315018315 0.3407821229050279
8 0.9884488448844885 0.8467153284671532 0.3016759776536313
9 0.9921259842519685 0.8081632653061225 0.30726256983240224
trn    0.992389
tst    0.816484
oot    0.325140
dtype: float64
CPU times: user 16.7 s, sys: 139 ms, total: 16.8 s
Wall time: 4.41 s


In [23]:
%%time
# NN

for niter in range(nitermax):  
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

    model = MLPClassifier()

    X_oot = X_oot_orig.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()

    model.fit(X_trn, Y_trn.values.ravel())   

    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['Fraud'] = Y_trn_save['Fraud']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['Fraud'] = Y_tst['Fraud']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions
    X_oot['Fraud'] = Y_oot['Fraud']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 0.7862969004893964 0.6741573033707865 0.5418994413407822
1 0.7885245901639344 0.7148148148148148 0.5027932960893855


/opt/conda/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


2 0.8273155416012559 0.7119341563786008 0.48044692737430167
3 0.7952380952380952 0.72 0.44692737430167595
4 0.7927631578947368 0.7316176470588235 0.5363128491620112
5 0.7752442996742671 0.7631578947368421 0.547486033519553
6 0.7867768595041322 0.7454545454545455 0.5195530726256983
7 0.7668308702791461 0.6937269372693727 0.4692737430167598
8 0.81421647819063 0.7432950191570882 0.4860335195530726
9 0.7948717948717948 0.77734375 0.5307262569832403
trn    0.792808
tst    0.727550
oot    0.506145
dtype: float64
CPU times: user 6min 42s, sys: 314 ms, total: 6min 43s
Wall time: 6min 43s


In [24]:
print('duration: ', datetime.now() - start_time)

duration:  0:10:19.196501


## The rest of the notebook makes the tables for your final model of choice. You need to run that final model only once (no CV). If you want you can run the below cell over and over by itself until it gives you a model you like. But you can't change from your best hyperparameters.

In [25]:
X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

# here's where you put your final model of choice
model = lgb.LGBMClassifier()

X_oot = X_oot_orig.copy()
X_trn_save = X_trn.copy()
Y_trn_save = Y_trn.copy()

model.fit(X_trn, Y_trn.values.ravel())   

predictions = model.predict_proba(X_trn_save)[:,1]
X_trn['predicted'] = predictions
X_trn['Fraud'] = Y_trn_save['Fraud']
topRows = int(round(X_trn.shape[0]*0.03))
temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
needed = temp.loc[:,'Fraud']
FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

predictions = model.predict_proba(X_tst)[:,1]
X_tst['predicted']=predictions
X_tst['Fraud'] = Y_tst['Fraud']
topRows = int(round(X_tst.shape[0]*0.03))
temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
needed = temp.loc[:,'Fraud']
FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

predictions = model.predict_proba(X_oot)[:,1]
X_oot['predicted']=predictions
X_oot['Fraud'] = Y_oot['Fraud']
topRows = int(round(X_oot.shape[0]*0.03))
temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
needed = temp.loc[:,'Fraud']
FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])

9 0.9949832775919732 0.7943262411347518 0.3016759776536313


In [26]:
X_trn_eval = X_trn.copy()
X_tst_eval = X_tst.copy()
X_oot_eval = X_oot.copy()

In [27]:
cols = ['bin','#recs','#g','#b','%g','%b','tot','cg','cb','%cg','FDR','KS','FPR']
FDR_trn = pd.DataFrame(np.zeros((101, 13)), columns = cols)
FDR_tst = pd.DataFrame(np.zeros((101, 13)), columns = cols)
FDR_oot = pd.DataFrame(np.zeros((101, 13)), columns = cols)
trn_sorted = X_trn_eval.sort_values('predicted',ascending=False)
tst_sorted = X_tst_eval.sort_values('predicted',ascending=False)
oot_sorted = X_oot_eval.sort_values('predicted',ascending=False)
bad_tot_trn = sum(X_trn_eval.loc[:, 'Fraud'])
bad_tot_tst = sum(X_tst_eval.loc[:, 'Fraud'])
bad_tot_oot = sum(X_oot_eval.loc[:, 'Fraud'])
num_tot_trn = len(X_trn_eval)
num_tot_tst = len(X_tst_eval)
num_tot_oot = len(X_oot_eval)
good_tot_trn = num_tot_trn - bad_tot_trn
good_tot_tst = num_tot_tst - bad_tot_tst
good_tot_oot = num_tot_oot - bad_tot_oot
oot_sorted.head(20)

,card_zip_total_3,card_state_max_30,card_state_total_3,card_state_max_14,card_state_total_7,card_state_total_0,card_zip_total_0,merch_state_avg_0,merch_state_avg_1,Merchnum_avg_0,merch_zip_avg_0,Merchnum_avg_1,merch_zip_avg_1,merch_zip_max_0,card_zip_avg_30,predicted,Fraud
94578,1.473094,4.259463,1.430944,4.733445,1.653427,-0.156522,-0.149090,-0.333307,-0.340786,-0.333333,-0.333280,-0.340816,-0.340757,-0.360515,0.051851,0.998860,1
94230,1.724612,4.259463,1.679227,4.733445,1.708525,2.208095,2.231405,7.646175,7.755561,7.645987,7.645856,7.755924,7.754082,5.760863,0.104172,0.996435,1
94884,1.994762,4.259463,1.945902,4.733445,2.157056,-0.121489,-0.113821,-0.212271,-0.217978,-0.212299,-0.212249,-0.218002,-0.217971,-0.267663,0.026231,0.996254,1
94595,1.552691,4.259463,1.509517,4.733445,1.729678,-0.033937,-0.025682,-0.093865,-0.097838,-0.093896,-0.093848,-0.097856,-0.097853,-0.176829,0.044060,0.996059,1
94610,1.565802,4.259463,1.522460,4.733445,1.742238,-0.013745,-0.005354,-0.447617,-0.456771,-0.447641,-0.447585,-0.456807,-0.456720,-0.448207,0.020429,0.995849,1
96396,0.792338,1.874173,0.758945,2.131862,0.697831,0.000240,0.008725,0.208287,0.208739,0.208249,0.208290,0.208736,0.208666,0.054963,0.723381,0.995829,0
95146,2.112660,4.259463,2.062283,4.733445,2.269999,0.060083,0.068969,-0.051173,-0.054521,-0.051205,-0.051158,-0.054537,-0.054544,-0.020494,0.028939,0.994913,1
91296,1.573410,4.876519,2.978145,5.406455,2.885120,4.715887,2.481504,8.694403,8.819141,8.694193,8.694038,8.819555,8.817463,6.564999,8.547195,0.994684,1
94713,1.880417,4.259463,1.833028,4.733445,2.043631,0.470786,0.482431,-0.393524,-0.401885,-0.393549,-0.393494,-0.401918,-0.401844,-0.360515,0.045005,0.992238,1
94636,1.868457,4.259463,1.821222,4.733445,2.032173,0.452366,0.463887,1.092970,1.106379,1.092914,1.092935,1.106419,1.106139,0.733638,0.067880,0.990295,1


In [28]:
for i in range(101):
    percent_rows_trn = int(round(X_trn_eval.shape[0]*0.01*i))
    percent_rows_tst = int(round(X_tst_eval.shape[0]*0.01*i))
    percent_rows_oot = int(round(X_oot_eval.shape[0]*0.01*i))
    temp_trn = trn_sorted.head(percent_rows_trn)
    temp_tst = tst_sorted.head(percent_rows_tst)
    temp_oot = oot_sorted.head(percent_rows_oot)
    num_bad_trn = sum(temp_trn.loc[:,'Fraud'])
    num_bad_tst = sum(temp_tst.loc[:,'Fraud'])
    num_bad_oot = sum(temp_oot.loc[:,'Fraud'])
    num_tot_trn = len(temp_trn)
    num_tot_tst = len(temp_tst)
    num_tot_oot = len(temp_oot)
    num_good_trn = num_tot_trn - num_bad_trn
    num_good_tst = num_tot_tst - num_bad_tst
    num_good_oot = num_tot_oot - num_bad_oot
    
    FDR_trn.loc[i, 'bin'] = i
    FDR_trn.loc[i,'#recs'] = 0
    FDR_trn.loc[i, 'tot'] = num_tot_trn
    FDR_trn.loc[i, 'cg'] = num_good_trn
    FDR_trn.loc[i, 'cb'] = num_bad_trn
    FDR_tst.loc[i, 'bin'] = i
    FDR_tst.loc[i, 'tot'] = num_tot_tst
    FDR_tst.loc[i, 'cg'] = num_good_tst
    FDR_tst.loc[i, 'cb'] = num_bad_tst
    FDR_oot.loc[i, 'bin'] = i
    FDR_oot.loc[i, 'tot'] = num_tot_oot
    FDR_oot.loc[i, 'cg'] = num_good_oot
    FDR_oot.loc[i, 'cb'] = num_bad_oot
    if i != 0:
        FDR_trn.loc[i, '#g'] = num_good_trn - FDR_trn.loc[i-1, 'cg']
        FDR_trn.loc[i, '#b'] = num_bad_trn - FDR_trn.loc[i-1, 'cb']
        FDR_trn.loc[i,'#recs'] = FDR_trn.loc[i, '#g'] + FDR_trn.loc[i, '#b']
        FDR_trn.loc[i, '%g'] = 100* (num_good_trn - FDR_trn.loc[i-1, 'cg']) / (num_tot_trn - FDR_trn.loc[i-1, 'tot'])
        FDR_trn.loc[i, '%b'] = 100 - FDR_trn.loc[i, '%g']
        FDR_trn.loc[i, '%cg'] = 100 * num_good_trn / good_tot_trn
        FDR_trn.loc[i, 'FDR'] = 100 * num_bad_trn / bad_tot_trn
        FDR_trn.loc[i, 'KS'] = FDR_trn.loc[i, 'FDR'] - FDR_trn.loc[i, '%cg']
        FDR_trn.loc[i, 'FPR'] = num_good_trn / num_bad_trn
        FDR_tst.loc[i, '#g'] = num_good_tst - FDR_tst.loc[i-1, 'cg']
        FDR_tst.loc[i, '#b'] = num_bad_tst - FDR_tst.loc[i-1, 'cb']
        FDR_tst.loc[i,'#recs'] = FDR_tst.loc[i, '#g'] + FDR_tst.loc[i, '#b']
        FDR_tst.loc[i, '%g'] = 100* (num_good_tst - FDR_tst.loc[i-1, 'cg']) / (num_tot_tst - FDR_tst.loc[i-1, 'tot'])
        FDR_tst.loc[i, '%b'] = 100 - FDR_tst.loc[i, '%g']
        FDR_tst.loc[i, '%cg'] = 100 * num_good_tst / good_tot_tst
        FDR_tst.loc[i, 'FDR'] = 100 * num_bad_tst / bad_tot_tst
        FDR_tst.loc[i, 'KS'] = FDR_tst.loc[i, 'FDR'] - FDR_tst.loc[i, '%cg']
        FDR_tst.loc[i, 'FPR'] = num_good_tst / num_bad_tst
        FDR_oot.loc[i, '#g'] = num_good_oot - FDR_oot.loc[i-1, 'cg']
        FDR_oot.loc[i, '#b'] = num_bad_oot - FDR_oot.loc[i-1, 'cb']
        FDR_oot.loc[i,'#recs'] = FDR_oot.loc[i, '#g'] + FDR_oot.loc[i, '#b']
        FDR_oot.loc[i, '%g'] = 100* (num_good_oot - FDR_oot.loc[i-1, 'cg']) / (num_tot_oot - FDR_oot.loc[i-1, 'tot'])
        FDR_oot.loc[i, '%b'] = 100 - FDR_oot.loc[i, '%g']
        FDR_oot.loc[i, '%cg'] = 100 * num_good_oot / good_tot_oot
        FDR_oot.loc[i, 'FDR'] = 100 * num_bad_oot / bad_tot_oot
        FDR_oot.loc[i, 'KS'] = FDR_oot.loc[i, 'FDR'] - FDR_oot.loc[i, '%cg']
        FDR_oot.loc[i, 'FPR'] = num_good_oot / num_bad_oot

FDR_trn

,bin,#recs,#g,#b,%g,%b,tot,cg,cb,%cg,FDR,KS,FPR
0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1,1.0,590.0,79.0,511.0,13.389831,86.610169,590.0,79.0,511.0,0.135246,85.451505,85.316259,0.154599
2,2.0,590.0,511.0,79.0,86.610169,13.389831,1180.0,590.0,590.0,1.010066,98.662207,97.652141,1.000000
3,3.0,590.0,585.0,5.0,99.152542,0.847458,1770.0,1175.0,595.0,2.011573,99.498328,97.486755,1.974790
4,4.0,590.0,587.0,3.0,99.491525,0.508475,2360.0,1762.0,598.0,3.016503,100.000000,96.983497,2.946488
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,96.0,590.0,590.0,0.0,100.000000,0.000000,56650.0,56052.0,598.0,95.959734,100.000000,4.040266,93.732441
97,97.0,590.0,590.0,0.0,100.000000,0.000000,57240.0,56642.0,598.0,96.969801,100.000000,3.030199,94.719064
98,98.0,590.0,590.0,0.0,100.000000,0.000000,57830.0,57232.0,598.0,97.979867,100.000000,2.020133,95.705686
99,99.0,590.0,590.0,0.0,100.000000,0.000000,58420.0,57822.0,598.0,98.989934,100.000000,1.010066,96.692308


In [29]:
FDR3.to_csv('FDR3.csv')

In [30]:
FDR_trn.to_csv('FDR_trn.csv', index=False)
FDR_tst.to_csv('FDR_tst.csv', index=False)
FDR_oot.to_csv('FDR_oot.csv', index=False)

In [31]:
print("duration: ", datetime.now() - start_time)

duration:  0:10:20.807194
